In [1]:
import pandas as pd
import os
absolutepath = os.path.abspath(os.getcwd())
fileDirectory = os.path.dirname(absolutepath)
parentDirectory = os.path.dirname(fileDirectory)
dataprocDirectory = os.path.join(fileDirectory, 'data/proc')
datarawDirectory = os.path.join(fileDirectory, 'data/raw')

/cluster/home/jiycai/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


# load affiliation list for each year

In [2]:
data_vec = pd.read_csv(os.path.join(dataprocDirectory, 'author_pool_affiliation_vectors_list_form.csv'))

In [3]:
data_vec.afid_masked_nunique.value_counts()

1      5585374
2      1315710
3       348681
4        90455
5        26200
        ...   
212          1
221          1
239          1
235          1
175          1
Name: afid_masked_nunique, Length: 171, dtype: int64

In [4]:
data_vec.head()

,authid_,year_,afid_masked_unique,afid_masked_nunique
0,6503846240,2007.0,[60009415],1
1,6503846240,2008.0,[60017482],1
2,6503846240,2009.0,[60017482],1
3,6503846240,2010.0,[60017482],1
4,6503846240,2011.0,[60017482],1


In [5]:
5585374/7393644

0.7554291226356044

# processs

In [6]:
def af_string_filter(af_string):
    af_list = af_string.replace("[",'').replace("]",'').split(' ')
    return af_list, len(af_list)

In [7]:
filtered_af = data_vec.afid_masked_unique.map(af_string_filter)

In [8]:
data_vec = data_vec.join(pd.DataFrame.from_records(filtered_af,
             columns = ['afid_masked_unique_filtered','afid_masked_nunique_filtered']))

In [9]:
data_vec_filtered_aff = data_vec[['authid_','year_','afid_masked_unique_filtered','afid_masked_nunique_filtered']]
data_vec_filtered_aff.columns = ['authid','cohort','afid_masked_unique_filtered','afid_masked_nunique_filtered']

# merge with panel and check fraction with only 1 af

In [10]:
balanced_dataset = pd.read_csv(os.path.join(dataprocDirectory,
                                                   'balanced_panel_wo_affiliation_never_treated_control_group.csv'))

In [11]:
balanced_dataset_authid_cohort = balanced_dataset[['authid','cohort']].drop_duplicates()

In [12]:
balanced_dataset_authid_cohort = balanced_dataset_authid_cohort.merge(data_vec_filtered_aff,
                                                                     how = 'left',
                                                                     on = ['authid','cohort'])

In [13]:
balanced_dataset_authid_cohort.afid_masked_nunique_filtered.value_counts()

1      685114
2      272962
3       69607
4       16672
5        4336
        ...  
68          1
129         1
101         1
96          1
66          1
Name: afid_masked_nunique_filtered, Length: 104, dtype: int64

In [14]:
balanced_dataset_authid_cohort.shape

(1051731, 4)

In [18]:
685114/1051731

0.6514156186325211

In [16]:
balanced_dataset_authid_cohort

,authid,cohort,afid_masked_unique_filtered,afid_masked_nunique_filtered
0,6503846240,2011,[60017482],1
1,6503867928,2011,[60025278],1
2,6503871635,2011,[60019616],1
3,6503888520,2011,[60023813],1
4,6503893958,2011,[60019499],1
...,...,...,...,...
1051726,57218719037,2017,[60033100],1
1051727,57218830100,2017,[60031150],1
1051728,57218842998,2017,"[60027363, 60015742]",2
1051729,57219137053,2017,[60013268],1


In [17]:
balanced_dataset_authid_cohort.to_csv(os.path.join(dataprocDirectory, 'author_cohort_jttp_af.csv'),
                                      index=False)